In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import pymysql

# import api keys
from api_keys import *

In [3]:
schema="gans"
host="wbs-project3-db.cwx6am7qyc91.eu-central-1.rds.amazonaws.com"
user="admin"
password=""
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [4]:
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']

# Fetching airports data from Gans db in AWS

In [5]:
airports = pd.read_sql('select * from airports', con)

# Calling arrival and weather data for the cities in our list

In [8]:
def get_flight_info(flight_json):
            # terminal
            try: terminal = flight_json['arrival']['terminal']
            except: terminal = None
            # aircraft
            try: aircraft = flight_json['aircraft']['model']
            except: aircraft = None

            return {
                'dep_airport':flight_json['departure']['airport']['name'],
                'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
                'terminal':terminal,
                'status':flight_json['status'],
                'aircraft':aircraft,
                'icao_code':airport_icoa
            }

for city in cities:
    
    globals()['%s_arrivals' % city] = pd.DataFrame()

    for i in range(len(airports[airports['municipality'] == city ]['icao_code'].values)):

        airport_icoa = airports[airports['municipality'] == city ]['icao_code'].values[i]
        to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
        from_local_time = (datetime.now() + timedelta(hours=9)).strftime('%Y-%m-%dT%H:00')
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

        querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

        headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': flight_api_key
            }

        response = requests.request("GET", url, headers=headers, params=querystring)

        arrivals = response.json()['arrivals']
        
        arrivals = pd.DataFrame([get_flight_info(flight) for flight in arrivals])

        globals()['%s_arrivals' % city] = globals()['%s_arrivals' % city].append(arrivals)


    # Weather data

    country = airports[airports['municipality'] == city ]['iso_country'].values[0]

    response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

    forecast_api = response.json()['list']

    weather_info = []

    for forecast_3h in forecast_api: 
        weather_hour = {}
        # datetime utc
        weather_hour['datetime'] = forecast_3h['dt_txt']
        # temperature 
        weather_hour['temperature'] = forecast_3h['main']['temp']
        # wind
        weather_hour['wind'] = forecast_3h['wind']['speed']
        # probability precipitation 
        try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
        except: weather_hour['prob_perc'] = 0
        # rain
        try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
        except: weather_hour['rain_qty'] = 0
        # wind 
        try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
        except: weather_hour['snow'] = 0
        weather_hour['municipality_iso_country'] = city + ',' + country
        weather_info.append(weather_hour)
        
    globals()['%s_weather' % city] = pd.DataFrame(weather_info)

    (
    globals()['%s_arrivals' % city]
        .replace({np.nan},'unknown')
        .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
        .to_sql(f'{city}_arrivals', if_exists='append', con=con, index=False)
    )

    globals()['%s_weather' % city].assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql(f'{city}_weather', if_exists='append', con=con, index=False)

